In [1]:
! pip install ..

Processing /Users/amiyaguchi/Research/wiki-forecast
  Created wheel for wiki-forecast: filename=wiki_forecast-0.3.2-cp36-none-any.whl size=19395 sha256=f77cbb37bedb4620fac1e9f963f70daff2f964cac417d72e020e6ba53445083f
  Stored in directory: /Users/amiyaguchi/Library/Caches/pip/wheels/4f/a1/40/0ace60fd57f0bb1ff87e264f714aa5255c9d115906476a9f13
Successfully built wiki-forecast
  Found existing installation: wiki-forecast 0.3.2
    Uninstalling wiki-forecast-0.3.2:
      Successfully uninstalled wiki-forecast-0.3.2


In [76]:
!tree -h ../data/clustered/sample_2_8_50

../data/clustered/sample_2_8_50
├── [   0]  _SUCCESS
├── [ 61M]  part-00000-da9ff087-c6ae-4dad-b9bd-d796a7ac0c15-c000.snappy.parquet
├── [ 61M]  part-00001-da9ff087-c6ae-4dad-b9bd-d796a7ac0c15-c000.snappy.parquet
├── [ 61M]  part-00002-da9ff087-c6ae-4dad-b9bd-d796a7ac0c15-c000.snappy.parquet
└── [ 61M]  part-00003-da9ff087-c6ae-4dad-b9bd-d796a7ac0c15-c000.snappy.parquet

0 directories, 5 files


In [36]:
from pyspark.sql import SparkSession, functions as F, Window
from graphframes import GraphFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wikicast.bipartition import induce_graph
from datetime import datetime, timedelta


def sign_to_string(*signs):
    result = ""
    for sign in signs:
        result += "x" if sign else "o"
    return result

def get_name(t): 
    return (start_date + timedelta(t)).strftime("%Y-%m-%d")


def transform(graph, pageviews, dates):
    sign = [f"sign_{x}" for x in range(8)]
    fiedler = [f"fiedler_{x}" for x in range(8)]     
    time_series = (
        pageviews.join(
            pageviews.groupBy("page_id").agg(F.sum("count").alias("total")).where("total >= 10").select("page_id"), 
            on="page_id",
            how="inner"
        )
        .withColumn("id", F.col("page_id"))
        .groupBy("id")
        .pivot("date")
        .agg(F.min("count"))
    )
    for date in set(dates) - set(time_series.columns):
        time_series = time_series.withColumn(date, F.lit(0))
    return (
        graph.vertices.join(graph.degrees, on="id", how="left")
        .join(graph.inDegrees, on="id", how="left")
        .join(graph.outDegrees, on="id", how="left")
        .join(time_series, on="id", how="inner")
        .withColumn("partition_id", F.udf(sign_to_string, "string")(*sign))
        .select("id", "title", "partition_id", "degree", "inDegree", "outDegree", *sign+fiedler+dates)
    )

def plot_degree(graph):
    degree = graph.inDegrees.groupBy("inDegree").count().orderBy(F.desc("count"))
    degree.show(n=5)

    df = degree.toPandas()
    plt.scatter(df["inDegree"], df["count"])
    plt.xscale("log")
    plt.yscale("log")



spark = SparkSession.builder.getOrCreate()
pages = spark.read.parquet("../data/clustered/sample_1_8_50")
pagelinks = spark.read.parquet("../data/enwiki/pagelinks")
pageviews = spark.read.parquet("../data/enwiki/pagecount_daily_v2")

pages.printSchema()
pagelinks.printSchema()

graph = induce_graph(GraphFrame(pages, pagelinks.selectExpr("from as src", "dest as dst")), relabel=False)
graph.cache()

print(f"graph has {graph.vertices.count()} articles and {graph.edges.count()} hyperlinks")
graph.vertices.show(vertical=True, n=1)

if False:
    plot_degree(graph)

start_date = datetime.strptime("2018-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2019-09-01", "%Y-%m-%d")
dates = [get_name(t) for t in range((end_date-start_date).days)]
data = transform(graph, pageviews, dates)
data.cache()

root
 |-- id: integer (nullable = true)
 |-- bias: boolean (nullable = true)
 |-- sign_0: boolean (nullable = true)
 |-- sign_1: boolean (nullable = true)
 |-- sign_2: boolean (nullable = true)
 |-- sign_3: boolean (nullable = true)
 |-- sign_4: boolean (nullable = true)
 |-- sign_5: boolean (nullable = true)
 |-- sign_6: boolean (nullable = true)
 |-- title: string (nullable = true)
 |-- is_redirect: boolean (nullable = true)
 |-- is_new: boolean (nullable = true)
 |-- fiedler_0: double (nullable = true)
 |-- fiedler_1: double (nullable = true)
 |-- fiedler_2: double (nullable = true)
 |-- fiedler_3: double (nullable = true)
 |-- fiedler_4: double (nullable = true)
 |-- fiedler_5: double (nullable = true)
 |-- fiedler_6: double (nullable = true)
 |-- fiedler_7: double (nullable = true)
 |-- sign_7: boolean (nullable = true)

root
 |-- from: integer (nullable = true)
 |-- dest: integer (nullable = true)

graph has 2936436 articles and 52893186 hyperlinks
-RECORD 0----------------------

DataFrame[id: int, title: string, partition_id: string, degree: int, inDegree: int, outDegree: int, sign_0: boolean, sign_1: boolean, sign_2: boolean, sign_3: boolean, sign_4: boolean, sign_5: boolean, sign_6: boolean, sign_7: boolean, fiedler_0: double, fiedler_1: double, fiedler_2: double, fiedler_3: double, fiedler_4: double, fiedler_5: double, fiedler_6: double, fiedler_7: double, 2018-01-01: bigint, 2018-01-02: bigint, 2018-01-03: bigint, 2018-01-04: bigint, 2018-01-05: bigint, 2018-01-06: bigint, 2018-01-07: bigint, 2018-01-08: bigint, 2018-01-09: bigint, 2018-01-10: bigint, 2018-01-11: bigint, 2018-01-12: bigint, 2018-01-13: bigint, 2018-01-14: bigint, 2018-01-15: bigint, 2018-01-16: bigint, 2018-01-17: bigint, 2018-01-18: bigint, 2018-01-19: bigint, 2018-01-20: bigint, 2018-01-21: bigint, 2018-01-22: bigint, 2018-01-23: bigint, 2018-01-24: bigint, 2018-01-25: bigint, 2018-01-26: bigint, 2018-01-27: bigint, 2018-01-28: bigint, 2018-01-29: bigint, 2018-01-30: bigint, 2018-01-31: 

In [93]:
%time data.repartitionByRange(1, "partition_id").write.parquet("../data/design_matrix/sample_1_8_50", mode="overwrite")

CPU times: user 3.73 ms, sys: 1.99 ms, total: 5.72 ms
Wall time: 29.2 s


In [35]:
!ls ../data/clustered

sample_10_8_50 sample_2_8_50  sample_4_8_50  sample_6_8_50  sample_8_8_50
sample_1_8_50  sample_3_8_50  sample_5_8_50  sample_7_8_50  sample_9_8_50


In [37]:
def transform_batch(spark, pagelinks, pageviews, dates):
    for i in range(10):
        print(f"iteration {i+1}")
        page = spark.read.parquet(f"../data/clustered/sample_{i+1}_8_50")
        graph = induce_graph(GraphFrame(pages, pagelinks.selectExpr("from as src", "dest as dst")), relabel=False)
        data = transform(graph, pageviews, dates)
        (
            data
            .repartitionByRange(1, "partition_id")
            .write
            .parquet(f"../data/design_matrix/sample_{i+1}_8_50", mode="overwrite")
        )

pagelinks.cache()
pageviews.cache()
transform_batch(spark, pagelinks, pageviews, dates)

iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
iteration {i+1}
